<a href="https://colab.research.google.com/github/1kaiser/PlanetaryBoundaryLayerProcessor/blob/main/PBLH_processor%F0%9F%92%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/1kaiser/PlanetaryBoundaryLayerProcessor.git

Cloning into 'PlanetaryBoundaryLayerProcessor'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 42 (delta 20), reused 17 (delta 6), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [2]:
%cd /content/PlanetaryBoundaryLayerProcessor

/content/PlanetaryBoundaryLayerProcessor


# Process to download

In [3]:
!npm install -g npm
!npm install puppeteer turndown bluebird
!apt-get update
!apt install chromium-chromedriver

/tools/node/bin/npx -> /tools/node/lib/node_modules/npm/bin/npx-cli.js
/tools/node/bin/npm -> /tools/node/lib/node_modules/npm/bin/npm-cli.js
+ npm@8.6.0
added 58 packages from 18 contributors, removed 292 packages and updated 142 packages in 10.034s

added 61 packages, and audited 62 packages in 45s

8 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 http

In [13]:
%%writefile index.js

'use strict';

const puppeteer = require('puppeteer');
const fs = require('fs');
var TurndownService = require('turndown');
const bluebird = require("bluebird");


(async() => {
  const browser = await puppeteer.launch({executablePath:"/usr/lib/chromium-browser/chromium-browser", args:['--no-sandbox']});
    let file = '\out.txt'
    if (!fs.existsSync(file)) {
        fs.writeFileSync(file, '', function (err) {
            if (err)
                console.log(err);
        });
    }
    var darray = fs.readFileSync('/content/PlanetaryBoundaryLayerProcessor/linkfiles/2021DD.txt', 'utf8').split('\n');
    let file1 = "MD2021"
    if (!fs.existsSync(file1)) {
        fs.mkdirSync(file1, function (err) {
            if (err)
                console.log(err);
        });
    }
    let withPage = (browser) => async (fn) => {
        const page = await browser.newPage();
        try {
            return await fn(page);
        } catch (err) {
            console.error(err.message);
            await page.close();
            return false;
        } finally {
            await page.close();
        }
    }
    const urls = [];
    const titles = [];
    for (let index = 0; index < darray.length; index++) {
        urls.push(darray[index].split('\,')[4]);
        titles.push(darray[index].split('\,')[5].replace('\r',''));
    }
    var outdarray = fs.readFileSync('\out.txt', 'utf8').split('\n');
    let count = 0
    bluebird.map(urls, async (url) => {
        if (!outdarray.join().includes(url)) {
            return withPage(browser)(async (page) => {
                await page._client.send("Page.setDownloadBehavior", {
                    behavior: "allow",
                    downloadPath: file1,
                });
                let completed = []
                let data = false;
                let attempts = 0;
                let viewSource
                while (data == false && attempts < 2) {
                    viewSource = await page.goto(url, {
                        waitUntil: "networkidle0",
                    });
                    data = await page.content();
                    attempts += 1;
                    if (data != false) {
                        fs.writeFile(`MD2021/${titles[urls.indexOf(url)]}`, await viewSource.buffer(), function (err) {
                            if (err) {
                                return console.log(err);
                            }

                            console.log("The file was saved!");
                        });
                        completed.push(url.concat('\n'))
                        fs.appendFileSync('out.txt', completed.toString())
                        // console.log(titles[urls.indexOf(url)])
                    }
                }
                console.log(urls.length - outdarray.length - count++, " <<remaining of ", urls.length - outdarray.length + 1, "with overall links", outdarray.length + urls.length - 1)
                if (count == urls.length - outdarray.length + 1) {
                    browser.close()
                }
            }).then((r) => ({
                result: r
            }), (e) => ({
                error: e
            }));
        }
    }, {
        concurrency: 7 //<< increase to go parallel tabs using bluebird library
    });
})()


Overwriting index.js


In [14]:
!node index.js

242  <<remaining of  243 with overall links 243
The file was saved!
241  <<remaining of  243 with overall links 243
The file was saved!
240  <<remaining of  243 with overall links 243
The file was saved!
239  <<remaining of  243 with overall links 243
The file was saved!
238  <<remaining of  243 with overall links 243
237  <<remaining of  243 with overall links 243
The file was saved!
The file was saved!
236  <<remaining of  243 with overall links 243
The file was saved!
235  <<remaining of  243 with overall links 243
The file was saved!
234  <<remaining of  243 with overall links 243
233  <<remaining of  243 with overall links 243
The file was saved!
The file was saved!
232  <<remaining of  243 with overall links 243
231  <<remaining of  243 with overall links 243
230  <<remaining of  243 with overall links 243
The file was saved!
The file was saved!
The file was saved!
229  <<remaining of  243 with overall links 243
The file was saved!
228  <<remaining of  243 with overall links 243


# creating video

In [ ]:
#create video 1
!ffmpeg \
-framerate 10 \
-pattern_type glob \
-i '/content/PlanetaryBoundaryLayerProcessor/MD/*.jpg' \
-vf 'pad=ceil(iw/2)*2:ceil(ih/2)*2' \
input1.mp4

In [ ]:
#create video 1
!ffmpeg \
-framerate 10 \
-pattern_type glob \
-i '/content/PlanetaryBoundaryLayerProcessor/MD2018/*.jpg' \
-vf 'pad=ceil(iw/2)*2:ceil(ih/2)*2' \
input0.mp4
#create vodeo 2
!ffmpeg \
-framerate 10 \
-pattern_type glob \
-i '/content/PlanetaryBoundaryLayerProcessor/MD2019/*.jpg' \
-vf 'pad=ceil(iw/2)*2:ceil(ih/2)*2' \
input1.mp4
#create vodeo 2
!ffmpeg \
-framerate 10 \
-pattern_type glob \
-i '/content/PlanetaryBoundaryLayerProcessor/MD2020/*.jpg' \
-vf 'pad=ceil(iw/2)*2:ceil(ih/2)*2' \
input2.mp4
#create vodeo 2
!ffmpeg \
-framerate 10 \
-pattern_type glob \
-i '/content/PlanetaryBoundaryLayerProcessor/MD2021/*.jpg' \
-vf 'pad=ceil(iw/2)*2:ceil(ih/2)*2' \
input3.mp4
#merge video 1 and video 2
!ffmpeg \
-i input0.mp4 -i input1.mp4 -i input2.mp4 -i input3.mp4 \
-filter_complex \
"[0:v][1:v]hstack=inputs=2[top];[2:v][3:v]hstack=inputs=2[bottom];[top][bottom]vstack=inputs=2[v]" \
finalOutput.mp4


In [16]:
#merge video 1 and video 2
!ffmpeg \
-i input0.mp4 -i input1.mp4 -i input2.mp4 -i input3.mp4 \
-filter_complex \
"[0:v][1:v]hstack=inputs=2[top];[2:v][3:v]hstack=inputs=2[bottom];[top][bottom]vstack=inputs=2[v]" \
-map "[v]" \
finalOutput.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

#zipping

In [ ]:
!zip -r /content/20220406.zip /content/PlanetaryBoundaryLayerProcessor/MD /content/PlanetaryBoundaryLayerProcessor/out.txt

  adding: content/PlanetaryBoundaryLayerProcessor/MD/ (stored 0%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210719_200.jpg (deflated 29%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210406_096.jpg (deflated 27%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210524_144.jpg (deflated 27%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210512_132.jpg (deflated 29%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210519_139.jpg (deflated 28%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210805_217.jpg (deflated 29%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210420_110.jpg (deflated 26%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210317_076.jpg (deflated 23%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210502_122.jpg (deflated 28%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210315_074.jpg (deflated 24%)
  adding: content/PlanetaryBoundaryLayerProcessor/MD/20210122_022.jpg (deflated 21%)